# Fleming Pipeline - ipython notebook version

Note on astrometry.net keys:
[Go to the website](http://nova.astrometry.net/api_help) and sign in and follow the instructions.

Need to add your own key to `~/.astroquery/config/astroquery.cfg` and `pipeline/astrometry_api_key.txt`

In [7]:
## Imports
import Reducer
import ShiftFinder
import FluxFinder
import DataAnalyser
import Constants
import Utilities
import MovingObjectFinder
import StreakFinder
import Cataloguer

from datetime import datetime
import os

#from astroquery import log
#log.setLevel("TRACE")


In [ ]:
## Reload imports (if files changed, only used for testing)
import importlib
importlib.reload(ShiftFinder)
importlib.reload(FluxFinder)
importlib.reload(DataAnalyser)
importlib.reload(Utilities)
importlib.reload(Cataloguer)


In [2]:
# REMOVE ME WHEN NOT USING DARK MODE
import matplotlib as mpl
COLOR = "white"
#COLOR = "black"
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR

In [ ]:

#r = Reducer.Reducer(Constants.folder,"No filter",Constants.file_name, 9, 50)
#r.reduce(True)
# # #set_size, n_sets = r.get_set_info()

In [8]:
c = Cataloguer.Cataloguer(Constants.workspace_dir, Constants.image_prefix, Constants.has_sets,
                          Constants.set_size, Constants.n_sets)
catalogue_image = os.path.join(Constants.workspace_dir, Constants.image_subdir,
        "{}_{:1}_{:03}{}".format(Constants.image_prefix, 1, 1, Constants.fits_extension))
c.catalogue(catalogue_image)
Utilities.print_job("cataloguing images")

Dumped 112 objects
Catalogues 1108 objects
starting job
Determining background stats


Finding sources
Found 1355 sources
 id      xcentroid      ...        flux                 mag          
---- ------------------ ... ------------------ ----------------------
 881 2078.6676678777644 ... 168.73045076485548     -5.567983666842608
 428 1312.5969976127772 ...  161.7241408363077     -5.521937131544891
1296  752.8349638637275 ... 158.32350036887271     -5.498863457765951
1121 1473.1473451113136 ...  133.2528962604471     -5.311691643050666
 606 1233.8588987377507 ...   131.428198146547     -5.296721384627498
 917 1015.1401707765954 ...  128.1301155178985    -5.2691280442307145
 787 1523.1761813437708 ... 118.90228311343414     -5.187975484613304
 368 1165.2002511192618 ... 112.79986988619136     -5.130771496730416
 ...                ... ...                ...                    ...
 630 1677.6664477527775 ... 1.0054829274069417  -0.005936752235173444
1315 1995.4931867732557 ... 1.0052621224314129  -0.005698297382724346
 594 1788.0072813668514 ... 1.0051012094980318  -0.0055

Finished cataloguing images at 18:20:03


In [9]:
sf = ShiftFinder.ShiftFinder(Constants.workspace_dir, Constants.image_prefix,
                             Constants.has_sets, Constants.set_size, Constants.n_sets)
sf.get_all_shifts()
Utilities.print_job("finding shifts")

Finding shifts in image: set 1; image 001


0.5129786036788362 0.730675845486819
Finding shifts in image: set 1; image 002


-1.4030576961818957 0.27367387952120126
Finding shifts in image: set 1; image 003


-0.973242785198444 -0.10824512685953902
Finding shifts in image: set 1; image 004


0.9722054543366667 -0.3853482223828335
Finding shifts in image: set 1; image 005


...

62.54757755829071 -30.036863275973992
Finding shifts in image: set 7; image 047


65.26593024178524 -29.850737536736737
Finding shifts in image: set 7; image 048


67.74028191692496 -29.15334395894548
Finding shifts in image: set 7; image 049


63.72981667857189 -29.430460724663458
Finding shifts in image: set 7; image 050


65.32827022203801 -28.73941531278992
Finished finding shifts at 18:22:40


In [10]:
ff = FluxFinder.FluxFinder(Constants.workspace_dir, Constants.image_prefix,
                           True, set_size=Constants.set_size, n_sets=Constants.n_sets)
ff.find_all_fluxes()
ff.make_light_curves()
Utilities.print_job("making light curves")

Set 1; image 001
Set 1; image 002
Set 1; image 003
Set 1; image 004
Set 1; image 005
Set 1; image 006
Set 1; image 007
Set 1; image 008
Set 1; image 009
Set 1; image 010
Set 1; image 011
Set 1; image 012
Set 1; image 013
Set 1; image 014
Set 1; image 015
Set 1; image 016
Set 1; image 017
Set 1; image 018
Set 1; image 019
Set 1; image 020
Set 1; image 021
Set 1; image 022
Set 1; image 023
Set 1; image 024
Set 1; image 025
Set 1; image 026
Set 1; image 027
Set 1; image 028
Set 1; image 029
Set 1; image 030
Set 1; image 031
Set 1; image 032
Set 1; image 033
Set 1; image 034
Set 1; image 035
Set 1; image 036
Set 1; image 037
Set 1; image 038
Set 1; image 039
Set 1; image 040
Set 1; image 041
Set 1; image 042
Set 1; image 043
Set 1; image 044
Set 1; image 045
Set 1; image 046
Set 1; image 047
Set 1; image 048
Set 1; image 049
Set 1; image 050
Set 2; image 001
Set 2; image 002
Set 2; image 003
Set 2; image 004
Set 2; image 005
Set 2; image 006
Set 2; image 007
Set 2; image 008
Set 2; image 0

Finding fluxes in image: set 2; image: 004
Finding fluxes in image: set 2; image: 005
Finding fluxes in image: set 2; image: 006
Finding fluxes in image: set 2; image: 007
Finding fluxes in image: set 2; image: 008
Finding fluxes in image: set 2; image: 009
Finding fluxes in image: set 2; image: 010
Finding fluxes in image: set 2; image: 011
Finding fluxes in image: set 2; image: 012
Finding fluxes in image: set 2; image: 013
Finding fluxes in image: set 2; image: 014
Finding fluxes in image: set 2; image: 015
Finding fluxes in image: set 2; image: 016
Finding fluxes in image: set 2; image: 017
Finding fluxes in image: set 2; image: 018
Finding fluxes in image: set 2; image: 019
Finding fluxes in image: set 2; image: 020
Finding fluxes in image: set 2; image: 021
Finding fluxes in image: set 2; image: 022
Finding fluxes in image: set 2; image: 023
Finding fluxes in image: set 2; image: 024
Finding fluxes in image: set 2; image: 025
Finding fluxes in image: set 2; image: 026
Finding flu

Finding fluxes in image: set 5; image: 045
Finding fluxes in image: set 5; image: 046
Finding fluxes in image: set 5; image: 047
Finding fluxes in image: set 5; image: 048
Finding fluxes in image: set 5; image: 049
Finding fluxes in image: set 5; image: 050
Finding fluxes in image: set 6; image: 001
Finding fluxes in image: set 6; image: 002
Finding fluxes in image: set 6; image: 003
Finding fluxes in image: set 6; image: 004
Finding fluxes in image: set 6; image: 005
Finding fluxes in image: set 6; image: 006
Finding fluxes in image: set 6; image: 007
Finding fluxes in image: set 6; image: 008
Finding fluxes in image: set 6; image: 009
Finding fluxes in image: set 6; image: 010
Finding fluxes in image: set 6; image: 011
Finding fluxes in image: set 6; image: 012
Finding fluxes in image: set 6; image: 013
Finding fluxes in image: set 6; image: 014
Finding fluxes in image: set 6; image: 015
Finding fluxes in image: set 6; image: 016
Finding fluxes in image: set 6; image: 017
Finding flu

In [11]:
r = None
c = None
sf = None

da = DataAnalyser.DataAnalyser(Constants.workspace_dir, Constants.image_prefix,
                               True, Constants.set_size, Constants.n_sets)

da.get_means_and_stds(False)
da.get_variables(False)
#da.output_results() #this should not be here - make it just create results folder with separate method
da.plot_means_and_stds("_adjusted")
ids = da.get_ids_for_avg()

ff.make_avg_curve(ids)
ff.divide_by_average()
avg_fname = "{}_avg{}".format(Constants.image_prefix, Constants.standard_file_extension)
ff.plot_light_curve(None, os.path.join(Constants.workspace_dir, avg_fname), True)
Utilities.print_job("adjusting light curves")

Finished adjusting light curves at 18:34:03


In [12]:
da.get_means_and_stds(True)
da.get_variables(True)
da.plot_means_and_stds("_adjusted")
da.output_results()
da.create_thumbnails(ff)

Utilities.print_job("everything")

cosmic detected at approx 10675s in id:
0534
cosmic detected at approx 11970s in id:
1158
cosmic detected at approx 11130s in id:
0314
cosmic detected at approx 10675s in id:
0581
cosmic detected at approx 11970s in id:
0360
cosmic detected at approx 12005s in id:
0091
cosmic detected at approx 12215s in id:
0507
cosmic detected at approx 12215s in id:
0233
cosmic detected at approx 11900s in id:
0277
cosmic detected at approx 10080s in id:
1018
cosmic detected at approx 11760s in id:
0849
cosmic detected at approx 10570s in id:
0892
cosmic detected at approx 11375s in id:
0609
cosmic detected at approx 12215s in id:
0939
cosmic detected at approx 10675s in id:
0988
cosmic detected at approx 12005s in id:
0274
cosmic detected at approx 12040s in id:
0721
cosmic detected at approx 11025s in id:
0661
cosmic detected at approx 11515s in id:
0903
cosmic detected at approx 11410s in id:
0399
cosmic detected at approx 11970s in id:
0776
cosmic detected at approx 11025s in id:
0117
cosmic det

Creating thumbnail for id 337.0, centroid 1264.5954,451.86992
Creating thumbnail for id 1096.0, centroid 1320.356,1428.2388
Creating thumbnail for id 742.0, centroid 1073.2756,972.04705
Finished everything at 18:35:11
